In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.display.max_columns= 30
pd.options.display.max_rows= 30
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows", None)
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from subprocess import check_output
df = pd.read_csv('123.csv')
eduDf=df[df['Type_code']== 'Education_status']
causesDf= df[df['Type_code']=='Causes']
meansDf= df[df['Type_code']== 'Means_adopted']
profDf=df[df['Type_code']=='Professional_profile']
socialDf=df[df['Type_code']=='Social_status']


: 

In [ ]:
df.shape 

: 

In [ ]:
df.info() 

: 

In [ ]:
df.describe() 

: 

In [ ]:
causesDf = df[df['Type_code']=='Causes']
causesDf.head()

: 

In [ ]:
print ('Date Collected from which states:',df['State'].unique())

: 

In [ ]:
causesDf = df[df['Type_code']== 'Causes']
causesDf.head()

: 

In [ ]:
meansDf = df[df['Type_code'] == 'Means_adopted']
meansDf.head()






: 

In [ ]:
 df.State.value_counts()

: 

In [ ]:
df= df.drop(df[(df.State== 'Total (Uts)') | (df.State == 'Total (All India)') | (df.State == 'Total (States)')].index)

: 

In [ ]:
df.State.value_counts()

: 

In [ ]:

state=df.groupby('State')['Total'].sum().to_frame()
state.sort_values(by= 'Total', ascending=True, inplace=True)
state.style.background_gradient(subset='Total', cmap='Blues')


: 

In [ ]:
f, ax = plt.subplots(1,2, figsize=(20,7))
df.groupby("State")["Total"].sum().sort_values(ascending = False)[:15].plot(kind= "bar",color ="green", ax= ax[0])
plt.ylabel("No. of Suicides")
df.groupby("State")["Total"].sum().sort_values(ascending = True) [:15].plot(kind ="bar", color= "orange", ax=ax[1])
plt.ylabel("No.of Suicides")
ax[0].title.set_text('Top 15 states with highest suicides')
ax[1].title.set_text('top 15 states with lowest suicides')


: 

In [ ]:

gender_suicide_counts=df.groupby('Gender')['Total'].sum().to_frame()

gender_suicide_counts.style.background_gradient(subset='Total', cmap='OrRd')




: 

In [ ]:
plt.figure(figsize=(8,8))
plt.pie(gender_suicide_counts['Total'], labels=gender_suicide_counts.index, autopct='%1.1f%%',colors=['azure','skyblue'], shadow= True)
plt.title('Total suicide by gender')

plt.show()


: 

In [ ]:
eduDf= eduDf[['Type','Gender','Total']]
edSort = eduDf.groupby(['Type','Gender'],as_index= False).sum().sort_values('Total',ascending=False)

: 

In [ ]:
causesDf= causesDf[['Type','Gender','Total']]
causes_Sort= causesDf.groupby(['Type','Gender'], as_index=False).sum().sort_values(by ='Total', ascending=False)


: 

In [ ]:
plt.figure(figsize=(18,12))
sns.barplot(data= causes_Sort, x='Type', y= 'Total', hue='Gender',palette='summer')
plt.xticks(rotation= 45, ha='right')
plt.tight_layout()

: 

In [ ]:
causesDf= df[df['Type_code']=='Causes']
group_year_wise= df.groupby(['State','Gender'],as_index=False).sum()
group_year_wisePvt = group_year_wise.pivot(index='Gender', columns= 'State', values='Total')
plt.figure(figsize=(14,6))
plt.xticks(rotation=45,ha ='right')
sns.heatmap(group_year_wisePvt,cmap='OrRd')
plt.tight_layout()


: 

In [ ]:
year=df.groupby('Year')['Total'].sum().to_frame()
state.sort_values(by= 'Total', ascending=True, inplace=True)
year.style.background_gradient(subset='Total',cmap='OrRd')

: 

In [ ]:

year.plot(kind='line', figsize=(10,5), color='blue')
plt.title('fd')
plt.xlabel('Year')
plt.ylabel('Total');

: 

In [ ]:
women=df[df['Gender']== 'Female']
women_year= women.groupby('State')['Total'].sum().to_frame()
women_year.rename(columns={'Total':'Female'}, inplace=True)




: 

In [ ]:
men=df[df['Gender']== 'Male']
men_year= men.groupby('State')['Total'].sum().to_frame()
men_year.rename(columns={'Total':'Male'}, inplace=True)




: 

In [ ]:
gender_year=men_year.join(women_year)
gender_year.style.background_gradient(cmap='OrRd', axis=0)


: 

In [ ]:
female_year=gender_year.drop(columns='Male')

female_year.style.background_gradient(cmap='OrRd', axis=0)


: 

In [ ]:
import geopandas as gpd
shp_gdf= gpd.read_file('/kaggle/input/india-gis-data/India States')
shp_gdf.set_index('st_nm', inplace =True)
shp_gdf.rename(index={'Andaman & Nicobar Island': 'A & N Islands','Arunanchal Pradesh': 'Arunachal Pradesh', 'Dadara & Nagar Havelli':'D & N Haveli','NCT of Delhi': 'Delhi (UT)'}, inplace=True)
shp_gdf.head()



: 

In [ ]:
merged= shp_gdf.join(female_year.iloc[:,0])
merged.at['Telangana','Female']= merged.at['Andhra Pradesh','Female']
merged.head()

: 

In [ ]:
fig, ax = plt.subplots (1,figsize=(12,12))
ax.axis('off')
ax.set_title('hjds',
fontdict={'fontsize': '15','fontweight': '3'})
fig= merged.plot(column='Female',cmap='Blues', linewidth=0.5, ax=ax, edgecolor='0.2',legend=True)
print(fig)



: 

In [ ]:
State1 = df[df['State']== 'West Bengal']

female= State1[State1['Gender']=='Female']
causes= female[female['Type_code']=='Causes']
female1= causes.drop(columns=['Year','Age_group','Type_code','Gender'])


causes= female1.groupby('Type').sum()['Total']
sort_causes= causes.sort_values(ascending=False)
causes_fig=sort_causes.plot(kind='bar', figsize=(14,6), title='gr',width=0.69)
causes_fig.set_xlabel('\nType')
causes_fig.set_ylabel('Total\n')
sns.set_style('whitegrid');


: 